In [1]:
import argparse
import fileinput
import nltk
import re
from collections import Counter
from unidecode import unidecode
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [8]:
parser = argparse.ArgumentParser()

parser.add_argument('files', metavar='FILE', nargs='*', help='files to read, if empty, stdin is used')
parser.add_argument('--outfile', default='./data/word_probs_new', help='output file')
args = parser.parse_args(args=[])

args.files = ['./data/dbpedia-2021-09-kewer/categories_lang=en_articles.ttl',
              './data/dbpedia-2021-09-kewer/redirects_lang=en_transitive.ttl',
              './data/dbpedia-2021-09-kewer/short-abstracts_lang=en.ttl']

print(args)

Namespace(files=['./data/dbpedia-2021-09-kewer/categories_lang=en_articles.ttl', './data/dbpedia-2021-09-kewer/redirects_lang=en_transitive.ttl', './data/dbpedia-2021-09-kewer/short-abstracts_lang=en.ttl'], outfile='./data/word_probs_new')


In [3]:
triples = set()

In [4]:
def normal_tokens(text):
    tokens = tokenizer.tokenize(text.lower())
    norm_tokens = []
    for token in tokens:
        norm_token = token
        if (not any(c.isalpha() for c in norm_token) and
                not (re.match(r'^[0-9]{4}$', norm_token) and
                    int(norm_token) < 2050)):
            norm_token = re.sub(r'\d', '0', norm_token)
        norm_tokens.append(unidecode(norm_token))
    return norm_tokens

In [5]:
word_counter = Counter()
total_count = 0

In [6]:
for line in fileinput.input(args.files):
    if not line.startswith('#'):
        subj, pred, obj = line.split(maxsplit=2)
        obj = obj[:obj.rfind('.')].strip()
        if obj.startswith('"'):
            lit_text = obj[obj.find('"')+1:obj.rfind('"')]
            lit_text = (lit_text.replace(r'\"', '"').replace(r'\t', '\t').replace(r'\b', '\b').
                replace(r'\n', '\n').replace(r'\r', '\r').replace(r'\f', '\f'))
            lit_text = lit_text.replace(':', '')
            sents = nltk.sent_tokenize(lit_text)
            for sent in sents:
                norm_tokens = normal_tokens(sent.lower())
                word_counter.update(norm_tokens)
                total_count += len(norm_tokens)

In [7]:
total_count

338437648

In [9]:
test_total_prob = 0.0
with open(args.outfile, 'w') as out_file:
    for word, count in sorted(word_counter.items()):
        print(word, count / total_count, sep='\t', file=out_file)
        test_total_prob += count / total_count

In [10]:
print('Total probability (should be 1):', test_total_prob)

Total probability (should be 1): 0.9999999999748292
